In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools

In [2]:
# почему то файл для скачивания XLS...а везде в ответах фигурируют CSV
# необходимо выбрать подходящую строчку для чтения файла

data = pd.read_csv('movie_bd_v5.xls')
# data = pd.read_csv('movie_bd_v5.csv')

# посмотрим что в файле прочитали (3 случайные строчки)
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1142,tt0317303,60000000,164433867,Daddy Day Care,Eddie Murphy|Jeff Garlin|Steve Zahn|Regina Kin...,Steve Carr,Who's your daddy?,Two men get laid off and have to become stay-a...,92,Comedy|Family,Columbia Pictures|Revolution Studios|Davis Ent...,5/3/2003,5.7,2003
595,tt0186589,11000000,13276953,Sugar & Spice,Marla Sokoloff|Marley Shelton|Melissa George|M...,Francine McDougall,Why would the most popular girl in school wear...,When Jack (James Marsden) and Diane (Marley Sh...,81,Crime|Comedy,New Line Cinema,1/24/2001,5.4,2001
1037,tt1758830,35000000,88058786,This Is 40,Paul Rudd|Leslie Mann|Jason Segel|Maude Apatow...,Judd Apatow,The sort-of sequel to 'Knocked Up',"Pete and Debbie are both about to turn 40, the...",134,Comedy,Apatow Productions,12/20/2012,5.5,2012


In [3]:
# посмотрим описание по нашим данным
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
# посмотрим на стобцы и типы данных в таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [6]:
# переформатируем столбец release_date в формат даты
data['release_date']=pd.to_datetime(data['release_date'],format="%m/%d/%Y")

# добавим новый столбец содержащий original_title и imdb_id в скобках 
# этот столбец я добавил, тк в ответах почему то присутствует именно в таком виде.
# и чтобы было проще сравнивать полученный мной ответ с вариантами я сделал столбец.
data['title_with_id']=data['original_title']+' ('+data['imdb_id']+')'

# добавим новый столбец с расчетом по прибыли/убытку по фильму
data['profit'] = data['revenue'] - data['budget']

In [7]:
# функция наименования месяца релиза фильма
# наверное можно и без дополнительного столбца, но если нам постоянно приходится использовать/рассчитывать, 
# то создание дополнительных столбцов оправдано - уменьшается последующий код 
def month_name(x):
    x=x.month
    if x==1:
        return 'Январь'
    if x==2:
        return 'Февраль'
    if x==3:
        return 'Март'
    if x==4:
        return 'Апрель'
    if x==5:
        return 'Май'
    if x==6:
        return 'Июнь'
    if x==7:
        return 'Июль'
    if x==8:
        return 'Август'
    if x==9:
        return 'Сентябрь'    
    if x==10:
        return 'Октябрь'    
    if x==11:
        return 'Ноябрь'
    if x==12:
        return 'Декабрь'
    else:
        return 'error'
    
# создаем новый столбец с названиями месяцев релиза фильма в исходных данных 
data['release_month'] = data.release_date.apply(month_name)

# это для проверки добавления нового столбца
#display(data.head(3))

In [8]:
# функция наименования сезона релиза фильма
# аналогично предыдущему, считаю правильнее добавить в таблицу столбец.
def season_name(x):
    x=x.month
    if x==1 or x==2 or x==12:
        return 'Зима'
    if 3<=x<=5:
        return 'Весна'
    if 6<=x<=8:
        return 'Лето'
    if 9<=x<=11:
        return 'Осень'
    else:
        return 'error'
    
# создаем новый столбец с названиями сезонов релиза фильма в исходных данных 
data['release_season'] = data.release_date.apply(season_name)

# это для проверки добавления нового столбца
#display(data.head(3))

In [9]:
# функция по подсчету количества вхождений жанров
# испольуется в вопросе 11
def spl(x):
    # делаем цикл по проверки вхождения каждого из жанров
    for genr in res:
        # смотрим, а равен ли входной жанр входному
        if genr in x:
            # если равен, то прибавляем единичку к сохраненному значению
            res[genr]+=1


In [10]:
# функция расчета длины названия фильма
# используется только 1 раз в вопросе 24.

def original_title_len(x):
    return len(x)

# создаем новый столбец с названиями месяцев релиза фильма в исходных данных 
data['movies_name_len'] = data.original_title.apply(original_title_len)

# проверяем результат добавления нового столбца
#display(data.head(3))

In [51]:
# функция составления комбинаций пар из актеров снимавшихся в одном фильме
# используется в впросе 27

def comb(x):
    # получаем на входе список актеров
    # обнуляем промежуточный список tred
    #c=0
    tred=[]
    for rex in itertools.combinations(x, 2):
        # проверяем на пару на уникальность и записываем каждую пару актеров в промежуточный список
        #if rex in uniq_dbl_list:
            tred.append(rex)
        #else:
        #    c+=1
            #print('не записываем ',rex)
    # возвращаем временный список на выход функции
    #print(c)
    return tred

## функция comb_v2(x) составления комбинаций пар из актеров снимавшихся в одном фильме используется в вопросе 27.   
## от её использования отказался, тк выдавал ошибку  
## вместо неё использую comb_v3(x)  

Сама ошибка
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
pandas\_libs\hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.map_locations()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'

In [55]:
# функцию не использую, оставил для истории.

def comb_v2(x):
    # получаем на входе список актеров
    # обнуляем промежуточный список tred

    tred=[]
    for rex in itertools.combinations(x, 2):
        # проверяем на пару на уникальность и записываем каждую пару актеров в промежуточный список
            rex=sorted(rex, reverse=False)
            if rex not in tred:
                tred.append(rex)

            #print('не записываем ',rex)

    return tred

In [56]:
# функция составления комбинаций пар из актеров снимавшихся в одном фильме
# используется в впросе 27

def comb_v3(x):
    # получаем на входе список актеров, сортируем его по возрастанию
    # обнуляем промежуточный список tred
    x=sorted(x, reverse=False)
    tred=[]
    
    # проходим циклом по комбинациям и записываем в список
    for rex in itertools.combinations(x, 2):
        tred.append(rex)
    return tred

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [13]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
# answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [14]:
# рассчитываем максимальный бюджет
max_budget=data['budget'].max()

# фильтруем исходные данные по максимальному бюджету
answer=data[(data.budget==max_budget)].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[1] = answer.to_string(index=False)

 Pirates of the Caribbean: On Stranger Tides (t...


ВАРИАНТ 2

In [15]:
# можно написать чуть короче, буз промежуточного расчета максимального бюджета
# первый вариант оправдан если бы мы и дальше использовали эту цифру.

# фильтруем исходные данные по максимальному бюджету
answer=data[(data.budget==data['budget'].max())].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[1] = answer.to_string(index=False)

 Pirates of the Caribbean: On Stranger Tides (t...


# 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
# answers['2'] = '...'

In [17]:
# рассчитываем максимальное значение длительности
max_runtime=data['runtime'].max()

# фильтруем исходные данные по максимальной длительности
answer=data[(data.runtime==max_runtime)].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[2] = answer.to_string(index=False)

 Gods and Generals (tt0279111)


In [18]:
# аналогично первому вопросу можно рассчитать и без промежуточной переменной

# фильтруем исходные данные по максимальной длительности
answer=data[(data.runtime==data['runtime'].max())].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[2] = answer.to_string(index=False)

 Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [19]:
# рассчитываем минимальное значение длительности фильма
min_runtime=data['runtime'].min()

# фильтруем исходные данные по минимальной длительности
answer=data[(data.runtime==min_runtime)].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[3] = answer.to_string(index=False)

 Winnie the Pooh (tt1449283)


In [20]:
# ВАРИАНТ 2
# фильтруем исходные данные по минимальной длительности
answer=data[(data.runtime==data['runtime'].min())].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[3] = answer.to_string(index=False)

 Winnie the Pooh (tt1449283)


# 4. Какова средняя длительность фильмов?


In [21]:
# рассчитываем среднюю длительность фильмов
mean_runtime=round(data['runtime'].mean(),0)

# выводим полученный результат на экран
display(mean_runtime)

# записываем наш ответ в словарь ответов
answers[4] = mean_runtime

110.0

# 5. Каково медианное значение длительности фильмов? 

In [22]:
# рассчитываем медианное значение длительности фильмов
median_runtime=round(data['runtime'].median(),0)

# выводим полученный результат на экран
display(median_runtime)

# записываем наш ответ в словарь ответов
answers[5] = median_runtime

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [23]:
# получение столбца profit вынесено в Предобработку

# рассчитываем максимальную прибыль по всем фильмам
max_profit=data['profit'].max()

# фильтруем исходные данные по максимальной прибыли
answer=data[(data.profit==max_profit)].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[6] = answer.to_string(index=False)

 Avatar (tt0499549)


# 7. Какой фильм самый убыточный? 

In [24]:
# рассчитываем минимальную прибыль (минимальная прибыль=максимальному убытку) по всем фильмам
min_profit=data['profit'].min()

# фильтруем исходные данные по минимальной прибыли
answer=data[(data.profit==min_profit)].title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[7] = answer.to_string(index=False)

 The Lone Ranger (tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
# применяем фильтр к данным по прибыльности и смотрим длину полученного среза
answer=len(data[data.profit>0])

# выводим полученный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[8] = answer

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [26]:
# применяем фильтр к данным, сортируем по убыванию, берём только первую строку 
# и ограничиваем выборку только названием фильма
answer=data[data.release_year==2008].sort_values('profit',ascending=False).head(1).title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[9] = answer.to_string(index=False)

 The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
# применяем фильтр по годам, сортируем по прибыли по возрастанию, 
# берем первую строку и ограничиваем выборку только названием фильма 
answer=data.query('2012<release_year<2015').sort_values('profit',ascending=True).head(1).title_with_id

# выводим полученный результат на экран
print(answer.to_string(index=False))

# записываем наш ответ в словарь ответов
answers[10] = answer.to_string(index=False)

 The Lone Ranger (tt1210819)


# 11. Какого жанра фильмов больше всего?

In [28]:
# скопируем исходные данные для работы
data2=data.copy()

# разделяем столбец Жанры по разделителю '|' и разбиваем на отдельные строчки
data2['genres'] = data2.genres.str.split('|').explode('genres')

# считаем количество строчек по жанрам, сортируем по убыванию и берем первую строчку
answer=data2['genres'].value_counts().sort_values(ascending=False).index[0]

# выведем результирующую табличку по жанрам (для сравнения с вариантом решения 2)
display(data2['genres'].value_counts().sort_values(ascending=False))

# выводим полученный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[11] = answer

Drama              283
Comedy             252
Action             225
Thriller           206
Adventure          164
Romance            122
Science Fiction    109
Crime              107
Family              94
Fantasy             84
Mystery             66
Horror              55
Animation           49
War                 25
Music               22
History             19
Western              5
Documentary          1
Foreign              1
Name: genres, dtype: int64

'Drama'

ВАРИАНТ 2

In [29]:
# копируем исходные данные
data2=data.copy()

# разделяем жанры по разделителю '|' и разбиваем на отдельные строчки
data2['genres'] = data2.genres.str.split('|').explode('genres')


# берем только столбец с жанрами
df=data2['genres']

# и создаем новый справочник с только уникальными названиями жанров, присваиваем значение 0 каждому из них
res={}

# проходим циклом по всем жанрам и заполняем справочник
for x in df:
    res[x]=0

# применяем функцию по подсчету количества вхождений
df=df.apply(spl)

# сортируем список (из нашего справочника) по убыванию значений
sorted_items=sorted(res.items(),key=lambda item: item[1], reverse=True)

# и возвращаем сортированный список в справочник
sorted_res={k: v for k, v in sorted_items}

# выводим финальный результат на экран
display(sorted_res)

# ну и правильный ответ на поставленный вопрос
display(next(iter(sorted_res)))

{'Drama': 283,
 'Comedy': 252,
 'Action': 225,
 'Thriller': 206,
 'Adventure': 164,
 'Romance': 122,
 'Science Fiction': 109,
 'Crime': 107,
 'Family': 94,
 'Fantasy': 84,
 'Mystery': 66,
 'Horror': 55,
 'Animation': 49,
 'War': 25,
 'Music': 22,
 'History': 19,
 'Western': 5,
 'Documentary': 1,
 'Foreign': 1}

'Drama'

Вариант 3

In [30]:
# копируем исходные данные
data2=data.copy()

# разделяем жанры по разделителю '|' и разбиваем на отдельные строчки
data2['genres'] = data2.genres.str.split('|').explode('genres')

# берем только столбец с жанрами
df=data2['genres']

# объявим новую переменную для создания списка вхождений жанров
# и используя функцию Counter посчитаем это число вхождений.
# most_common() - сделает сортировку по убыванию.
res=Counter(df).most_common(),

# выведем на экран получившийся список для проверки решения с другими вариантами
display(res)

# нашим ответом будет первая строка списка
res[0]

([('Drama', 283),
  ('Comedy', 252),
  ('Action', 225),
  ('Thriller', 206),
  ('Adventure', 164),
  ('Romance', 122),
  ('Science Fiction', 109),
  ('Crime', 107),
  ('Family', 94),
  ('Fantasy', 84),
  ('Mystery', 66),
  ('Horror', 55),
  ('Animation', 49),
  ('War', 25),
  ('Music', 22),
  ('History', 19),
  ('Western', 5),
  ('Documentary', 1),
  ('Foreign', 1)],)

[('Drama', 283),
 ('Comedy', 252),
 ('Action', 225),
 ('Thriller', 206),
 ('Adventure', 164),
 ('Romance', 122),
 ('Science Fiction', 109),
 ('Crime', 107),
 ('Family', 94),
 ('Fantasy', 84),
 ('Mystery', 66),
 ('Horror', 55),
 ('Animation', 49),
 ('War', 25),
 ('Music', 22),
 ('History', 19),
 ('Western', 5),
 ('Documentary', 1),
 ('Foreign', 1)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
# В этом блоке задача решена в соответствии с постановкой.
# в следующем, то что подразумевалось надо сделать

# копируем исходные данные
data2=data.copy()
# разделяем столбец Жанры для подсчета
data2['genres'] = data2.genres.str.split('|')
df=data2.explode('genres')

# считаем общее количество фильмов по жанрам, сколько было прибыльных и сколько убыточных
number_total=df.groupby(['genres'])['profit'].count().sort_values(ascending=False)
number_profit=df[(df.profit>0)].groupby(['genres'])['profit'].count().sort_values(ascending=False)
number_loss=df[(df.profit<0)].groupby(['genres'])['profit'].count().sort_values(ascending=False)

# объединяем 3 списка в один
df2=pd.concat([number_total,number_profit,number_loss],axis=1)
# переименовываем названия столбцов для удобства
df2.columns=['total','profit','loss']
# заменяем значения NaN на 0
df2=df2.fillna(0)

# считаем частоту прибыльности жанров
df2['freq_prof']=(df2['total']-df2['loss'])/df2['total']

# получаем сортированный список
df3=df2.sort_values('freq_prof',ascending=False)

# и первая запись и есть искомое
answer=df3['freq_prof'].index[0]

# выводим финальный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[12] = answer

'Documentary'

In [32]:
# после уточнения задачи в slaсk, задача была "чуть-чуть" изменена...надо сделать следующее:

# Нужно сделать фильтрацию фильмов по прибыли > 0, 
# и затем просто посчитать какой жанр встречается чаще среди полученного датасета

# копируем исходные данные, сразу с фильтрацией по прибыли
data2=data[(data.profit>0)].copy()

# разделяем столбец Жанры для подсчета
data2['genres'] = data2.genres.str.split('|')
df=data2.explode('genres')

# группируем по жанрам, считаем число фильмов по жанрам и сортируем по убыванию
df2=df.groupby(['genres'])['revenue'].count().sort_values(ascending=False)

# наш ответ будет в верхней строке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[12] = answer

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
# копируем исходные данные
data2=data.copy()

# разделяем столбец Режисер для подсчета
data2['director'] = data2.director.str.split('|')
df=data2.explode('director')

# группируем по режисерам, суммируем по выручке, сортируем по убыванию. Первая строка - наш ответ.
answer=df.groupby(['director'])['revenue'].sum().sort_values(ascending=False).index[0]

# выводим финальный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[13] = answer

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [34]:
# копируем исходные данные
data2=data.copy()

# разделяем столбец Жанры для подсчета
data2['genres'] = data2.genres.str.split('|')
df=data2.explode('genres')

# оставляем только интересующий нас жанр
df=df[(df.genres=='Action')]

# разделяем режисеров для подсчета
df['director'] = df.director.str.split('|')
df2=df.explode('director')

# группируем по директору, считаем количество снятых фильмов и сортируем список по убыванию количества фильмов
df3=df2.groupby(['director'])['director'].count().sort_values(ascending=False)

# ответом будет первый в списке
answer=df3.index[0]

# выводим финальный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[14] = answer

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [35]:
# копируем исходные данные с нужным нам годом
data2=data[data.release_year==2012].copy()

# разделяем столбец Актеры для подсчета
data2['cast'] = data2.cast.str.split('|')
df=data2.explode('cast')

# группируем по актеру, считаем сумму кассовых сборов по всем фильмам 
# и сортируем по убыванию
df2=df.groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

# ответом будет первый в списке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем наш ответ в словарь ответов
answers[15] = answer

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [36]:
# Примечание: в фильмах, где бюджет выше среднего по данной выборке.

# посчитаем размер среднего бюджета
budget_mean=data.budget.mean()

# копируем исходные данные только с выше или равным среднему бюджету
data2=data[data.budget>=budget_mean].copy()

# разделяем столбец Актеры для подсчета
data2['cast'] = data2.cast.str.split('|')
df=data2.explode('cast')

# группируем по актеру, считаем количество строчек и сортируем список по убыванию 
df2=df.groupby(['cast'])['director'].count().sort_values(ascending=False)

# ответом будет первый в списке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[16] = answer

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [37]:
# сохраняем копию данных (сразу применяем фильтр по актеру)
data2=data[data.cast.str.contains('Nicolas Cage')].copy()

# разбиваем столбец жанр на строки
data2['genres'] = data2.genres.str.split('|')
df=data2.explode('genres')

# группируем по жанрам, считаем число фильмов и сортируем по убыванию
df2=df.groupby(['genres'])['revenue'].count().sort_values(ascending=False)

# наш ответ будет в верхней строке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[17] = answer

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [38]:
# сохраняем копию данных (сразу применяем фильтр по киностудии) в новую переменную
data2=data[data.production_companies.str.contains('Paramount Pictures')].copy()

# сортируем список фильмов по фозрастанию прибыли
data2=data2.sort_values('profit',ascending=True)

# наш ответ будет в верхней строке
answer=data2.title_with_id.iloc[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[18] = answer

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [39]:
# копируем исходные данные
data2=data.copy()

# группируем по году выпуска фильма, считаем число фильмов и сортируем по убыванию
df=data2.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

# наш ответ будет в верхней строке
answer=df.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[19] = answer

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [40]:
# сохраняем копию данных (сразу применяем фильтр по киностудии)
data2=data[data.production_companies.str.contains('Warner Bros')].copy()

# группируем по году выпуска фильма, считаем число фильмов и сортируем по убыванию
df=data2.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

# наш ответ будет в верхней строке
answer=df.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[20] = answer

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [41]:
# группируем по году выпуска фильма, считаем число фильмов и сортируем по убыванию
df=data.groupby(['release_month'])['profit'].count().sort_values(ascending=False)

# наш ответ будет в верхней строке
answer=df.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[21] = answer

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [42]:
# сохраняем копию данных (сразу применяем фильтр по сезону) в новую переменную
data2=data[data.release_season=='Лето'].copy()

# группируем по сезону выпуска фильма и считаем число фильмов
df=data2.groupby(['release_season'])['profit'].count()

# наш ответ будет в верхней строке
answer=df.iloc[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[22] = answer

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [43]:
# сохраняем копию данных (сразу применяем фильтр по сезону)
data2=data[data.release_season=='Зима'].copy()

# разделяем режисеров для подсчета
df['director'] = data2.director.str.split('|')
df=data2.explode('director')

# группируем по директору, считаем количество снятых фильмов и сортируем список по убыванию количества фильмов
df2=df.groupby(['director'])['director'].count().sort_values(ascending=False)

# ответом будет первый в списке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[23] = answer

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [44]:
# сохраняем копию данных в новую переменную
data2=data.copy()

# разделяем столбец Киностудии для подсчета
data2['production_companies'] = data2.production_companies.str.split('|')
df=data2.explode('production_companies')

# группируем по киностудии, считаем среднюю длину названий фильмов и сортируем список по убыванию 
df2=df.groupby(['production_companies'])['movies_name_len'].mean().sort_values(ascending=False)
#display(df2)

# ответом будет первый в списке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[24] = answer

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [45]:
# сохраняем копию данных в новую переменную
data2=data.copy()
data2['overview_len']=data2['overview'].apply(len)

# разделяем столбец Киностудии для подсчета
data2['production_companies'] = data2.production_companies.str.split('|')
df=data2.explode('production_companies')

# группируем по киностудии, считаем среднюю длину названий фильмов и сортируем список по убыванию 
df2=df.groupby(['production_companies'])['overview_len'].mean().sort_values(ascending=False)
#display(df2)

# ответом будет первый в списке
answer=df2.index[0]

# выводим финальный результат на экран
display(answer)

# записываем результат в словарь
answers[25] = answer

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [46]:
# фильтруем фильмы по рейтингу через квартиль, и выводим один столбец
answer=data[data['vote_average']>data['vote_average'].quantile(q=0.99)].original_title

# выводим финальный результат на экран 
display(answer)

# записываем результат в словарь
answers[26] = 'Inside Out, The Dark Knight, 12 Years a Slave'

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [52]:
# копируем исходные данные
df = data.copy()

# разбиваем столбец Актеры по разделителю '|'
df['cast'] = df.cast.str.split('|')

# для каждой строки расчитываем пары, через собственную функцию comb 
# и записываем список пар актеров на место списка актеров
df['cast']=df.cast.apply(comb)

# разбиваем список пар на отдельные строчки
df1 = df.explode('cast')

# выводим список пар актеров отсортированных по убыванию количества фильмов
display(df1.cast.value_counts().sort_values(ascending=False))

# запысываем результат в словарь
answers[27] = 'Daniel Radcliffe, Rupert Grint'

(Daniel Radcliffe, Emma Watson)         8
(Daniel Radcliffe, Rupert Grint)        8
(Rupert Grint, Emma Watson)             7
(Johnny Depp, Helena Bonham Carter)     6
(Ben Stiller, Owen Wilson)              6
                                       ..
(Kurt Russell, Ving Rhames)             1
(Steve Carell, Olivia Wilde)            1
(Jean Dujardin, Penelope Ann Miller)    1
(Brendan Fraser, Timothy Dalton)        1
(Sandra Bullock, Scoot McNairy)         1
Name: cast, Length: 18121, dtype: int64

ВАРИАНТ 2

In [58]:
# копируем исходные данные
df = data.copy()

# разбиваем столбец Актеры по разделителю '|'
df['cast'] = df.cast.str.split('|')

# для каждой строки расчитываем пары, через собственную функцию comb 
# и записываем список пар актеров на место списка актеров
# здесь я использую модифицированную функцию, в которой сортирую 
# по алфавиту актеров до составления комбинаций
df['cast']=df.cast.apply(comb_v3)

# разбиваем список пар на отдельные строчки
df1 = df.explode('cast')

# выводим список пар актеров отсортированных по убыванию количества фильмов
display(df1.cast.value_counts().sort_values(ascending=False))

# записываем результат в словарь
answers[27] = 'Daniel Radcliffe, Rupert Grint'

(Daniel Radcliffe, Emma Watson)          8
(Daniel Radcliffe, Rupert Grint)         8
(Emma Watson, Rupert Grint)              8
(Helena Bonham Carter, Johnny Depp)      6
(Ben Stiller, Owen Wilson)               6
                                        ..
(Catherine Zeta-Jones, Nick Chinlund)    1
(Alexandra Daddario, Marilyn Burns)      1
(Amy Poehler, Sigourney Weaver)          1
(Bruce McGill, Orlando Bloom)            1
(Laurie Holden, Sean Bean)               1
Name: cast, Length: 17943, dtype: int64

# Submission

In [59]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{1: ' Pirates of the Caribbean: On Stranger Tides (t...',
 2: ' Gods and Generals (tt0279111)',
 3: ' Winnie the Pooh (tt1449283)',
 4: 110.0,
 5: 107.0,
 6: ' Avatar (tt0499549)',
 7: ' The Lone Ranger (tt1210819)',
 8: 1478,
 9: ' The Dark Knight (tt0468569)',
 10: ' The Lone Ranger (tt1210819)',
 11: 'Drama',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker (tt0267626)',
 19: 2015,
 20: 2014,
 21: 'Сентябрь',
 22: 450,
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: 'Inside Out, The Dark Knight, 12 Years a Slave',
 27: 'Daniel Radcliffe, Rupert Grint'}

In [60]:
# добавляем в словарь ответов "+" при получении правильного ответа

# копируем исходный словарь ответов
answers2=answers.copy()

# пробегаемся циклом и добавляем "+" во все, тк я правильно решил все поставленные задачи :))))
for i in range(1,28):
    answers2[i]=[answers2[i],'+']
    
# выводим новый список на экран
answers2

{1: [' Pirates of the Caribbean: On Stranger Tides (t...', '+'],
 2: [' Gods and Generals (tt0279111)', '+'],
 3: [' Winnie the Pooh (tt1449283)', '+'],
 4: [110.0, '+'],
 5: [107.0, '+'],
 6: [' Avatar (tt0499549)', '+'],
 7: [' The Lone Ranger (tt1210819)', '+'],
 8: [1478, '+'],
 9: [' The Dark Knight (tt0468569)', '+'],
 10: [' The Lone Ranger (tt1210819)', '+'],
 11: ['Drama', '+'],
 12: ['Drama', '+'],
 13: ['Peter Jackson', '+'],
 14: ['Robert Rodriguez', '+'],
 15: ['Chris Hemsworth', '+'],
 16: ['Matt Damon', '+'],
 17: ['Action', '+'],
 18: ['K-19: The Widowmaker (tt0267626)', '+'],
 19: [2015, '+'],
 20: [2014, '+'],
 21: ['Сентябрь', '+'],
 22: [450, '+'],
 23: ['Peter Jackson', '+'],
 24: ['Four By Two Productions', '+'],
 25: ['Midnight Picture Show', '+'],
 26: ['Inside Out, The Dark Knight, 12 Years a Slave', '+'],
 27: ['Daniel Radcliffe, Rupert Grint', '+']}

In [61]:
# и убедиться что ни чего не пропустил)
display(len(answers))
display(len(answers2))

27

27